In [1]:
from __future__ import print_function

import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

In [3]:
logs_path = './tf_logs/'

In [4]:
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

In [5]:
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

In [6]:
# Construct model and encapsulating all ops into scopes, makeing TensorBoard's Graph visualization more convenient
with tf.name_scope('Model'):
    pred = tf.nn.softmax(tf.matmul(x, W) + b)
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

init = tf.global_variables_initializer()

tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', acc)
merged_summary_op = tf.summary.merge_all()

In [9]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c, summary = sess.run([optimizer, cost, merged_summary_op], feed_dict={x:batch_xs, y:batch_ys})
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c/total_batch
        
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
            
    print ("Optimization Finished.")

    print ("Accuracy: ", acc.eval({x: mnist.test.images, y: mnist.test.labels}))


Epoch: 0001 cost= 1.183215916
Epoch: 0002 cost= 0.665157872
Epoch: 0003 cost= 0.552687355
Epoch: 0004 cost= 0.498539627
Epoch: 0005 cost= 0.465495737
Epoch: 0006 cost= 0.442601094
Epoch: 0007 cost= 0.425489646
Epoch: 0008 cost= 0.412197126
Epoch: 0009 cost= 0.401349456
Epoch: 0010 cost= 0.392335561
Epoch: 0011 cost= 0.384799051
Epoch: 0012 cost= 0.378163470
Epoch: 0013 cost= 0.372340971
Epoch: 0014 cost= 0.367315574
Epoch: 0015 cost= 0.362723090
Epoch: 0016 cost= 0.358591370
Epoch: 0017 cost= 0.354894234
Epoch: 0018 cost= 0.351498925
Epoch: 0019 cost= 0.348334718
Epoch: 0020 cost= 0.345384747
Epoch: 0021 cost= 0.342742135
Epoch: 0022 cost= 0.340207627
Epoch: 0023 cost= 0.337910023
Epoch: 0024 cost= 0.335722090
Epoch: 0025 cost= 0.333652167
Optimization Finished.
Accuracy:  0.9135


In [11]:
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.WARN)

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz


IOError: [Errno socket error] [Errno 60] Operation timed out

In [13]:
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/tmp/tensorflow_logs/example'

In [14]:
n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

x = tf.placeholder(tf.float32, [None, 784], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

In [15]:
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    tf.summary.histogram('relu1', layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    tf.summary.histogram('relu2', layer_2)
    
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    
    return out_layer

In [16]:
weights = {
    'w1' : tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2' : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3' : tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3'),
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

In [17]:
# Encapsulating all ops into scopes, making TensorBoard's Graph Visualisation more convenient
with tf.name_scope('Model'):
    pred = multilayer_perceptron(x, weights, biases)
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(acc, tf.float32))

In [18]:
init = tf.global_variables_initializer()

tf.summary.scalar("loss", loss)
tf.summary.scalar("accuracy", acc)
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
for grad, var in grads:
    tf.summary.histogram(var.name + '/gradient', grad)
merged_summary_op = tf.summary.merge_all()

ValueError: Tried to convert 'values' to a tensor and failed. Error: None values not supported.

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c, summary = sess.run([apply_grads, loss, merged_summary_op], feed_dict={x:batch_xs, y:batch_ys})
            summary_writer.add_summary(summary, epoch*total_batch + i)
            avg += c/total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print('Optimization Finished.')
    
    print ("Accuracy: ", acc.eval({x:mnist.test.images, y: mnist.test.labels}))
            